In [32]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [33]:
df=pd.read_csv(r"A:\Work Docs\DATA Set\titanic_train.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [34]:
df.isnull().mean()*100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

In [35]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'], inplace=True)
df.sample(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
536,0,1,male,45.0,0,0,26.5500,S
66,1,2,female,29.0,0,0,10.5000,S
511,0,3,male,NaN,0,0,8.0500,S
819,0,3,male,10.0,3,2,27.9000,S
29,0,3,male,NaN,0,0,7.8958,S


In [36]:
x=df.drop(columns=['Survived'])
y=df['Survived']

In [37]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2,random_state=2)

In [38]:
x_train.sample(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
405,2,male,34.0,1,0,21.0000,S
308,2,male,30.0,1,0,24.0000,C
491,3,male,21.0,0,0,7.2500,S
686,3,male,14.0,4,1,39.6875,S
361,2,male,29.0,1,0,27.7208,C


### Pipeline-1

Preparing Pipelines

In [39]:
numerical_features=['Age','Fare']
# numerical_transformer=Pipeline(steps=[(),()])
numerical_transformer=Pipeline(steps=[('imputer',SimpleImputer(strategy='median')),('scaler',StandardScaler())])


Categorical_features=['Embarked','Sex']
# categorical_transformer=Pipeline(steps=[(),()])
categorical_transformer=Pipeline(steps=[('imputer', SimpleImputer( strategy='most_frequent')),( 'ohe', OneHotEncoder( handle_unknown='ignore'))])


Preparing Column Transformer

In [40]:
# preprocessor=ColumnTransformer(transformers=[(),(),(),()])
preprocessor=ColumnTransformer(transformers=[('num',numerical_transformer,numerical_features),('cat',categorical_transformer,Categorical_features)])

### Pipeline-2

In [41]:
# clf=Pipeline(steps=[(),()])
clf=Pipeline(steps=[('preprocessor',preprocessor),('classifier',LogisticRegression())])

In [42]:
from sklearn import set_config

set_config(display='diagram')
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked', 'Sex'])])),
                ('classifier', LogisticRegression())])

In [54]:
parameter_grid={
'preprocessor__num__imputer__strategy':['mean','median'],
'preprocessor__cat__imputer__strategy': ['most_frequent', 'constant'],
'classifier__C': [0.1,1.0,10,100]
}

grid_search= GridSearchCV(clf,parameter_grid, cv=10)

In [55]:
grid_search.fit(x_train,y_train)



GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Age',
                                                                          'Fare']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Embarked',
                                                                          'Sex'])])),
                                       ('classifier', LogisticRegression())]),
             param_grid={'classifier__C': [0.1, 1.0, 10, 100],
                         'preprocessor__cat__imputer__strategy': ['most_frequent',
                                                                  'constant'],
                         'preprocessor__num__imputer__strategy': ['mean',
                                                                  'median']})

In [56]:
print(f"Best parameters")
print(grid_search.best_params_)

Best parameters
{'classifier__C': 1.0, 'preprocessor__cat__imputer__strategy': 'most_frequent', 'preprocessor__num__imputer__strategy': 'mean'}


In [57]:
print(f'Internal CV score: {grid_search.best_score_: .3f}')

Internal CV score:  0.788


In [58]:
import pandas as pd
cv_results =pd.DataFrame(grid_search.cv_results_)
cv_results=cv_results.sort_values("mean_test_score",ascending=False)
cv_results[['param_classifier__C','param_preprocessor__cat__imputer__strategy','param_preprocessor__num__imputer__strategy','mean_test_score']]


,param_classifier__C,param_preprocessor__cat__imputer__strategy,param_preprocessor__num__imputer__strategy,mean_test_score
4,1.0,most_frequent,mean,0.787852
5,1.0,most_frequent,median,0.787852
6,1.0,constant,mean,0.787852
7,1.0,constant,median,0.787852
8,10,most_frequent,mean,0.787852
9,10,most_frequent,median,0.787852
10,10,constant,mean,0.787852
11,10,constant,median,0.787852
12,100,most_frequent,mean,0.787852
13,100,most_frequent,median,0.787852
